# Mount Google Drive and installing requirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd 'drive/MyDrive/SC_GPT'

/content/drive/MyDrive/SC_GPT


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 116.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 93.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

# Fetch and unzip the checkpoint

In [4]:
!git lfs install
!git clone https://huggingface.co/metehergul/scgpt

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'scgpt'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), 538.98 KiB | 1.09 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/SC_GPT/scgpt/.git/hooks/post-checkout': Permission denied


# Training

In [5]:
!export CUDA_VISIBLE_DEVICES=0
!python train.py --output_dir=MODEL_SAVE_PATH --model_type=gpt2 --model_name_or_path='./scgpt' --do_train --do_eval --eval_data_file=data/taxi/train.txt --per_gpu_train_batch_size 1 --num_train_epochs 20 --learning_rate 1e-5 --overwrite_cache --use_tokenize --train_data_file=data/taxi/train.txt --overwrite_output_dir

2023-07-17 06:38:04.957295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 06:38:06.233419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/17/2023 06:38:10 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/17/2023 06:38:35 - INFO - __main__ -   Training/evaluation parameters Namespace(train_data_file='data/taxi/train.txt', output_dir='MODEL_SAVE_PATH', eval_data_file='data/taxi/train.txt', model_type='gpt2', model_name_or_path='./scgpt', mlm=False, mlm_probability=0.15, config_name='', tokenizer_name='', cache_dir='', block_size=80, do_train=True, do_eval=True, evaluate_during_training=False, do_lower_case=False, per_gp

# Decoding

In [6]:
!export CUDA_VISIBLE_DEVICES=0
# !python generate.py --model_type=gpt2 --model_name_or_path=MODEL_SAVE_PATH --num_samples 5 --input_file=data/restaurant/test.txt --top_k 5 --output_file=results.json --length 80
!python generate.py --model_type=gpt2 --model_name_or_path=MODEL_SAVE_PATH --num_samples 5 --input_file=data/taxi/test.txt --top_p 0.9 --output_file=results.json --length 80

07/17/2023 06:42:25 - INFO - __main__ -   Namespace(model_type='gpt2', model_name_or_path='MODEL_SAVE_PATH', prompt='', padding_text='', xlm_lang='', length=80, num_samples=5, temperature=1.0, repetition_penalty=1.0, top_k=0, top_p=0.9, no_cuda=False, seed=42, stop_token=None, input_file='data/taxi/test.txt', output_file='results.json', nc=1, use_token=False, device=device(type='cuda'), n_gpu=1)
  0% 0/47 [00:00<?, ?it/s]2023-07-17 06:42:36.827355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/17/2023 06:42:38 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.
100% 47/47 [05:15<00:00,  6.71s/it]


# Evaluate

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
!python evaluator.py --domain taxi --target_file results.json

##############################################
BLEU SCORE & SLOT ERROR on GENERATED SENTENCES
##############################################
Metric       :	BLEU	T.ERR	A.ERR
HDC          :	1.0000	0.00%	0.00%
Ref          :	1.0000	0.70%	0.30%
----------------------------------------------
This Model   :	0.1384	3.70%	1.80%
FIELNAME: results.json, BLEU: 0.1384151912112596, ERR:3.6973131736526946


# Interact

In [ ]:
!python interact.py --model_type=gpt2 --model_name_or_path=MODEL_SAVE_PATH --length 50 --num_samples 5

07/12/2023 17:45:33 - INFO - __main__ -   Namespace(model_type='gpt2', model_name_or_path='MODEL_SAVE_PATH', prompt='', padding_text='', xlm_lang='', length=50, num_samples=5, temperature=1.0, repetition_penalty=1.0, top_k=0, top_p=0.9, no_cuda=False, seed=42, stop_token=None, device=device(type='cuda'), n_gpu=1)
Model prompt >>> 
^C
